# 编写自定义文件存储

```python
# settings.py

 # 设置Django的文件存储类
DEFAULT_FILE_STORAGE='utils.fdfs.storage.FDFSStorage'

# 设置fdfs使用的client.conf文件路径
FDFS_CLIENT_CONF='./utils/fdfs/client.conf'

# 设置fdfs存储服务器上nginx的IP和端口号
FDFS_URL='http://172.16.179.131:8888/'

```

```python
from django.core.files.storage import Storage
from django.conf import settings
from fdfs_client.client import Fdfs_client


class FDFSStorage(Storage):
    '''fast dfs文件存储类'''
    def __init__(self, client_conf=None, base_url=None):
        '''初始化'''
        if client_conf is None:
            client_conf = settings.FDFS_CLIENT_CONF
        self.client_conf = client_conf

        if base_url is None:
            base_url = settings.FDFS_URL
        self.base_url = base_url

    def _open(self, name, mode='rb'):
        '''打开文件时使用'''
        pass

    def _save(self, name, content):
        '''保存文件时使用'''
        # name:你选择上传文件的名字
        # content:包含你上传文件内容的File对象

        # 创建一个Fdfs_client对象
        client = Fdfs_client(self.client_conf)

        # 上传文件到fast dfs系统中
        res = client.upload_by_buffer(content.read())

        # dict
        # {
        #     'Group name': group_name,
        #     'Remote file_id': remote_file_id,
        #     'Status': 'Upload successed.',
        #     'Local file name': '',
        #     'Uploaded size': upload_size,
        #     'Storage IP': storage_ip
        # }
        if res.get('Status') != 'Upload successed.':
            # 上传失败
            raise Exception('上传文件到fast dfs失败')

        # 获取返回的文件ID
        filename = res.get('Remote file_id')

        return filename

    def exists(self, name):
        '''Django判断文件名是否可用'''
        return False

    def url(self, name):
        '''返回访问文件的url路径'''
        return self.base_url+name
```

```python
# 获取首页分类商品展示信息
for type in types: # GoodsType
    # 获取type种类首页分类商品的图片展示信息
    image_banners = IndexTypeGoodsBanner.objects.filter(type=type, display_type=1).order_by('index')
    # 获取type种类首页分类商品的文字展示信息
    title_banners = IndexTypeGoodsBanner.objects.filter(type=type, display_type=0).order_by('index')

    # 动态给type增加属性，分别保存首页分类商品的图片展示信息和文字展示信息
    type.image_banners = image_banners
    type.title_banners = title_banners
```

# 使用Redis缓存

- 配置

```python
# setting.py

# Django的缓存配置
CACHES = {
    "default": {
        "BACKEND": "django_redis.cache.RedisCache",
        "LOCATION": "redis://172.16.179.142:6379/9",
        "OPTIONS": {
            "CLIENT_CLASS": "django_redis.client.DefaultClient",
        }
    }
}

# 配置session存储
SESSION_ENGINE = "django.contrib.sessions.backends.cache"
SESSION_CACHE_ALIAS = "default"
```

- 读取数据

```python
from django_redis import get_redis_connection

# 获取用户购物车中商品的数目
user = request.user
cart_count = 0
if user.is_authenticated():
    # 用户已登录
    conn = get_redis_connection('default')
    cart_key = 'cart_%d'%user.id
    cart_count = conn.hlen(cart_key)
```

# 指定类型

- a:Httprequest：指定输入参数的类型
- class A -> Httpresponse：指定输出的类型